In [1]:
import time
import numpy as np
from tqdm.notebook import tqdm

from components.dataset_jo import *

dataset = Dataset_subjectDependent('data')
segment_lenght = 5 #second
dataset.set_segment(7680//(128*segment_lenght))
for filename in dataset.get_file_list():
    data, labels, _ = dataset.get_data(filename, return_type='numpy')
    print(filename, labels.shape, labels.sum(axis=0)/labels.shape[0]*100)
    break

Found: 32 files
s01 (480, 2) [47.5 60. ]


In [2]:
def train_model(X_ori,y_ori,groups_ori,filename=None, kernel='rbf',return_text=False):
    # Make a copy because I am paranoid
    X,y,groups = X_ori.copy(), y_ori.copy(), groups_ori.copy()

    from sklearn.svm import SVC
    from sklearn.model_selection import StratifiedShuffleSplit 
    from sklearn.model_selection import cross_val_score

    model = SVC(kernel=kernel,max_iter=50000)
    cv = StratifiedShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
    cross = cross_val_score(model, X, y, cv=cv, n_jobs=8)
    
    # We probably dont need this
    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X, y)
    ans = model.predict(X)
    acc = sum(ans == y) / len(y)
    # If the model answer with all 0 or 1, we print this message
    text = None
    if( sum(ans) == len(y) or sum(ans) == 0 ): 
        text = f"-----WARNING: Model {filename} failed to learn: sum(ans)={sum(ans)} sum(y)={sum(y)} len(y)={len(y)}"
    if(return_text):
        return model, acc, cross, text
    else:
        if(text != None): print(text)
        return model, acc, cross

## 2. Spectral Features

In [3]:
from mne_features.feature_extraction import FeatureExtractor

In [4]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})


accs,cv_means,cv_stds  = [],[],[]
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

accs_v,cv_means_v,cv_stds_v  = [],[],[]
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.67917|10-CV=0.6225|STD=0.01346|Time spend=31.954869747161865
	AROUSAL-s02|Acc=0.62083|10-CV=0.60167|STD=0.01965|Time spend=0.562838077545166
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=96.0 len(y)=480
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=0.48596811294555664
	AROUSAL-s04|Acc=0.68125|10-CV=0.63667|STD=0.02667|Time spend=0.514404296875
	AROUSAL-s05|Acc=0.59167|10-CV=0.52917|STD=0.03966|Time spend=0.5258924961090088
	AROUSAL-s06|Acc=0.62917|10-CV=0.60833|STD=0.03801|Time spend=0.5145721435546875
	AROUSAL-s07|Acc=0.68125|10-CV=0.6375|STD=0.01304|Time spend=0.5113976001739502
	AROUSAL-s08|Acc=0.67917|10-CV=0.60417|STD=0.02181|Time spend=0.5185661315917969
	AROUSAL-s09|Acc=0.62292|10-CV=0.6025|STD=0.00382|Time spend=0.5433547496795654
	AROUSAL-s10|Acc=0.7|10-CV=0.64417|STD=0.04519|Time spend=0.5769264698028564
-----WARNING: Model s11 failed to learn: sum(ans)=0.0 sum(y)=180.0 len(y)=480
	AROUSAL-s11|Acc=0.625|10-CV=0.625|STD=0.0|Time spend=0.57

## 3. Asymetry

In [5]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")

left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


In [6]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tAROUSAL-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs[feature_name].append(acc)
        cv_means[feature_name].append(cross.mean())
        cv_stds[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"AROUSAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

accs_v,cv_means_v,cv_stds_v  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tVALENCE-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs_v[feature_name].append(acc)
        cv_means_v[feature_name].append(cross.mean())
        cv_stds_v[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"VALENCE-{feature_name}|Acc={sum(accs_v[feature_name])/len(accs_v[feature_name])}|10-CV={sum(cv_means_v[feature_name])/len(cv_means_v[feature_name])}|STD={sum(cv_stds_v[feature_name])/len(cv_stds_v[feature_name])}")

for feature_name in ['DASM','RASM','DCAU']:
    accs[feature_name].extend(accs_v[feature_name])
    cv_means[feature_name].extend(cv_means_v[feature_name])
    cv_stds[feature_name].extend(cv_stds_v[feature_name])
    print(f"TOTAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

	AROUSAL-s01-DASM|Acc=0.60417|10-CV=0.6|STD=0.00745|Time spend=0.3191564083099365
	AROUSAL-s01-RASM|Acc=0.61458|10-CV=0.59833|STD=0.01106|Time spend=0.3558821678161621
	AROUSAL-s01-DCAU|Acc=0.62292|10-CV=0.59917|STD=0.01261|Time spend=0.38868188858032227
	AROUSAL-s02-DASM|Acc=0.61875|10-CV=0.59917|STD=0.02022|Time spend=0.2985725402832031
	AROUSAL-s02-RASM|Acc=0.60833|10-CV=0.59917|STD=0.0025|Time spend=0.334439754486084
	AROUSAL-s02-DCAU|Acc=0.62917|10-CV=0.60667|STD=0.01893|Time spend=0.3689405918121338
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=96.0 len(y)=480
	AROUSAL-s03-DASM|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=0.29027366638183594
	AROUSAL-s03-RASM|Acc=0.80208|10-CV=0.8|STD=0.0|Time spend=0.3195760250091553
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=96.0 len(y)=480
	AROUSAL-s03-DCAU|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=0.3520383834838867
	AROUSAL-s04-DASM|Acc=0.65208|10-CV=0.6025|STD=0.02529|Time spend=0.3154008388519287
	AROUSAL-s04-RASM|Acc=0

## 4. Connectivity

### 4.1 $ \text{PCC}_{\text{time}}(i,j) = \frac{\text{Cov}[\mathbf{X}_i, \mathbf{X}_j]}{\sigma_{\mathbf{X}_i} \sigma_{\mathbf{X}_j}} $

In [7]:

def pearson_correlation(x,y):
    """ x,y denoted the signal_x and signal_y following the equation """
    cov = np.cov(x, y)
    # print(cov)
    # [[ 8806859.74527069  8007149.0906219 ] ==> [[cov_xx, cov_xy]
    # [ 8007149.0906219  10396797.72458848]]      [cov_yx, cov_yy]]
    cov_xy = cov[0,1] # or cov[1,0]
    cov_xx = cov[0,0]
    cov_yy = cov[1,1]
    corr = cov_xy / ( cov_xx**0.5 * cov_yy**0.5  )
    return corr

def _cal(p_id, partial_data):
    # print(f"p_id:{p_id} - data to run {partial_data.shape}")
    from itertools import combinations
    pcc = []
    for index in range(partial_data.shape[0]):
        pcc_epoch = []
        for comb in combinations(list(range(partial_data.shape[1])), 2):
            pcc_ab = pearson_correlation(partial_data[index, comb[0], :], partial_data[index, comb[1], :]   )
            pcc_epoch.append(pcc_ab)
        pcc_epoch = np.hstack(pcc_epoch)
        pcc.append(pcc_epoch)
    pcc = np.vstack(pcc)
    return pcc

def calculate_pcc(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(_cal, [p_id, data[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    except Exception as e:
        print(e)
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)

In [8]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    data_csd = mne.preprocessing.compute_current_source_density(data)
    pcc = calculate_pcc(data_csd.get_data())
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.70625|10-CV=0.63083|STD=0.02142|Time spend=2.7023706436157227
	VALENCE-s01|Acc=0.63958|10-CV=0.56|STD=0.02734|Time spend=3.0101306438446045
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.6|10-CV=0.6|STD=0.0|Time spend=2.429328441619873
	VALENCE-s02|Acc=0.58333|10-CV=0.56167|STD=0.02614|Time spend=2.7316009998321533
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=2.4666967391967773
	VALENCE-s03|Acc=0.55|10-CV=0.55|STD=0.0|Time spend=2.7770159244537354
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15

In [9]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.70625|10-CV=0.63083|STD=0.02142|Time spend=2.7023706436157227
	VALENCE-s01|Acc=0.63958|10-CV=0.56|STD=0.02734|Time spend=3.0101306438446045
	AROUSAL-s02|Acc=0.6|10-CV=0.6|STD=0.0|Time spend=2.429328441619873
-----WARNING: Model s02 failed to learn: sum(ans)=480.0 sum(y)=288.0 len(y)=480
	VALENCE-s02|Acc=0.58333|10-CV=0.56167|STD=0.02614|Time spend=2.7316009998321533
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=2.4666967391967773
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=96.0 len(y)=480
	VALENCE-s03|Acc=0.55|10-CV=0.55|STD=0.0|Time spend=2.7770159244537354
-----WARNING: Model s03 failed to learn: sum(ans)=480.0 sum(y)=264.0 len(y)=480
	AROUSAL-s04|Acc=0.6|10-CV=0.6|STD=0.0|Time spend=2.401986837387085
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=192.0 len(y)=480
	VALENCE-s04|Acc=0.6|10-CV=0.6|STD=0.0|Time spend=2.6966781616210938
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=192.0 len(y)=480
	AROUSAL-s05|Acc=0.558

### 4.2 $ \text{PCC}_{freq} (i,j) = \frac{\text{Cov}[\hat{\mathbf{X}}_i, \hat{\mathbf{X}}_j]}{\sigma_{\hat{\mathbf{X}}_i} \sigma_{\hat{\mathbf{X}}_j}} $

In [10]:
def calculate_fft(signal, sfreq):
    """ signal: can be 1D array of (n_sample,) or 2D array of (n_signal, n_sample)  """
    number_sample = signal.shape[-1]

    # the result will be a complex number. We can obtain the magnitude using `absolute`
    magnitude = np.abs(np.fft.fft(signal))
    # scale the result
    magnitude = magnitude / (number_sample/2)
    # Selecting the range
    magnitude = magnitude.T[:number_sample//2].T
    freq_range = np.fft.fftfreq(number_sample, d=1/sfreq)[:number_sample//2]

    return magnitude, freq_range

In [11]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)
    magnitude, freq_range = calculate_fft(data_csd.get_data(), 128)
    pcc = calculate_pcc(magnitude)
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.7875|10-CV=0.67667|STD=0.0238|Time spend=3.006192445755005
	VALENCE-s01|Acc=0.82917|10-CV=0.59667|STD=0.05286|Time spend=3.3221335411071777
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.64583|10-CV=0.63333|STD=0.01936|Time spend=2.7718424797058105
	VALENCE-s02|Acc=0.61667|10-CV=0.5725|STD=0.03229|Time spend=3.1048591136932373
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=2.47969913482666
	VALENCE-s03|Acc=0.65208|10-CV=0.5725|STD=0.02529|Time spend=2.8198533058166504
Fitted sphere radius:         95.3 mm
Origin head coordi

In [12]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.7875|10-CV=0.67667|STD=0.0238|Time spend=3.006192445755005
	VALENCE-s01|Acc=0.82917|10-CV=0.59667|STD=0.05286|Time spend=3.3221335411071777
	AROUSAL-s02|Acc=0.64583|10-CV=0.63333|STD=0.01936|Time spend=2.7718424797058105
	VALENCE-s02|Acc=0.61667|10-CV=0.5725|STD=0.03229|Time spend=3.1048591136932373
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=2.47969913482666
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=96.0 len(y)=480
	VALENCE-s03|Acc=0.65208|10-CV=0.5725|STD=0.02529|Time spend=2.8198533058166504
	AROUSAL-s04|Acc=0.61875|10-CV=0.59667|STD=0.00667|Time spend=2.64009165763855
	VALENCE-s04|Acc=0.61042|10-CV=0.59583|STD=0.00672|Time spend=2.9409255981445312
	AROUSAL-s05|Acc=0.65833|10-CV=0.57|STD=0.04628|Time spend=2.744375467300415
	VALENCE-s05|Acc=0.61667|10-CV=0.60333|STD=0.01067|Time spend=3.047403335571289
	AROUSAL-s06|Acc=0.675|10-CV=0.61333|STD=0.04365|Time spend=2.8756020069122314
	VALENCE-s06|Acc=0.75|10-CV=0.75|STD=0.0|Time spend=3.198224

### 4.3 $ \text{PLV}(j,k) = \frac{1}{T} | \Sigma^{T}_{t=1} e^{i(\phi^{t}_{j} - \phi^{t}_{k})}   | $

In [13]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLV_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    print(phase.shape)
    from itertools import combinations
    plv = []
    # count = 0
    for index in range(phase.shape[0]):
        plv_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            plv_ab = np.abs(np.average(np.exp(complex(0,1) * phase_diff), axis=1))
            plv_epoch.append(plv_ab)
        plv_epoch = np.vstack(plv_epoch)
        # print(plv_epoch.shape) => (300, 32, 65, 23)
        plv_epoch_5 = np.concatenate([ plv_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        plv.append(np.expand_dims(plv_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    plv = np.vstack( plv )
    return plv.squeeze()

def calculate_plv(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLV_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [14]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    plv = calculate_plv(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(plv, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(plv, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
========= close ========
	AROUSAL-s01|Acc=0.82917|10-CV=0.70417|STD=0.0301|Time spend=21.25208020210266
	VALENCE-s01|Acc=0.83542|10-CV=0.61583|STD=0.04448|Time spend=22.633060693740845
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
(60, 32, 65, 108)
========= close ========
	AROUSAL-s02|Acc=0.69375|10-CV=0.65917|STD=0.03128|Time spend=21.170554399490356
	VALENCE-s02|Acc=0.7375|10-CV=0.71417|STD=0.03652|Time spend=22.291282653808594
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 

In [15]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.82917|10-CV=0.70417|STD=0.0301|Time spend=21.25208020210266
	VALENCE-s01|Acc=0.83542|10-CV=0.61583|STD=0.04448|Time spend=22.633060693740845
	AROUSAL-s02|Acc=0.69375|10-CV=0.65917|STD=0.03128|Time spend=21.170554399490356
	VALENCE-s02|Acc=0.7375|10-CV=0.71417|STD=0.03652|Time spend=22.291282653808594
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=22.833285570144653
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=96.0 len(y)=480
	VALENCE-s03|Acc=0.76875|10-CV=0.7025|STD=0.05297|Time spend=24.05602788925171
	AROUSAL-s04|Acc=0.72083|10-CV=0.67167|STD=0.03379|Time spend=20.20174241065979
	VALENCE-s04|Acc=0.73333|10-CV=0.67417|STD=0.03717|Time spend=21.34722590446472
	AROUSAL-s05|Acc=0.6875|10-CV=0.61167|STD=0.02915|Time spend=19.658122062683105
	VALENCE-s05|Acc=0.63958|10-CV=0.59917|STD=0.00786|Time spend=20.926229238510132
	AROUSAL-s06|Acc=0.625|10-CV=0.6025|STD=0.03765|Time spend=22.4258291721344
	VALENCE-s06|Acc=0.75|10-CV=0.75|STD=0.0|Time spend=23.52

### 4.4 $ \text{PLI}(j,k) =  \frac{1}{T} | \Sigma^{T}_{t=1} \text{sign}(Im[e^{i (\phi^{t}_{j} - \phi^{t}_{k})}])  | $

In [16]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLI_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    from itertools import combinations
    pli = []
    # count = 0
    for index in range(phase.shape[0]):
        pli_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            pli_ab = np.abs(np.average(   np.sign(np.imag(   np.exp(complex(0,1) * phase_diff) ))  , axis=1))
            pli_epoch.append(pli_ab)
        pli_epoch = np.vstack(pli_epoch)
        pli_epoch_5 = np.concatenate([ pli_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        pli.append(np.expand_dims(pli_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    pli = np.vstack( pli )
    return pli.squeeze()

def calculate_pli(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLI_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [17]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    pli = calculate_pli(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(pli, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pli, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.93542|10-CV=0.60833|STD=0.01708|Time spend=21.347578287124634
	VALENCE-s01|Acc=0.99792|10-CV=0.52|STD=0.0344|Time spend=22.788733959197998
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.87083|10-CV=0.61083|STD=0.02299|Time spend=21.382967948913574
	VALENCE-s02|Acc=0.93333|10-CV=0.58833|STD=0.02843|Time spend=23.179449319839478
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.84167|10-CV=0.8|STD=0.0|Time spend=20.61544966697693
	VALENCE-s03|Acc=0.9875|10-CV=0.53167|STD=0.03114|Time spend=22.888082027435303
Fitted sphere radius:         95.3 mm
Origin head c

In [18]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.93542|10-CV=0.60833|STD=0.01708|Time spend=21.347578287124634
	VALENCE-s01|Acc=0.99792|10-CV=0.52|STD=0.0344|Time spend=22.788733959197998
	AROUSAL-s02|Acc=0.87083|10-CV=0.61083|STD=0.02299|Time spend=21.382967948913574
	VALENCE-s02|Acc=0.93333|10-CV=0.58833|STD=0.02843|Time spend=23.179449319839478
	AROUSAL-s03|Acc=0.84167|10-CV=0.8|STD=0.0|Time spend=20.61544966697693
	VALENCE-s03|Acc=0.9875|10-CV=0.53167|STD=0.03114|Time spend=22.888082027435303
	AROUSAL-s04|Acc=0.85|10-CV=0.59583|STD=0.01003|Time spend=25.853066205978394
	VALENCE-s04|Acc=0.80208|10-CV=0.59833|STD=0.00333|Time spend=27.641732454299927
	AROUSAL-s05|Acc=0.84375|10-CV=0.52333|STD=0.0263|Time spend=22.116210460662842
	VALENCE-s05|Acc=0.73333|10-CV=0.58667|STD=0.01247|Time spend=23.979090452194214
	AROUSAL-s06|Acc=0.96667|10-CV=0.57833|STD=0.02303|Time spend=22.677961826324463
	VALENCE-s06|Acc=0.84792|10-CV=0.75|STD=0.0|Time spend=24.429141521453857
	AROUSAL-s07|Acc=0.95|10-CV=0.625|STD=0.0|Time spend=

## 5. CSP

In [19]:
from components.CSP import CSP
from mne_features.feature_extraction import FeatureExtractor

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

In [20]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape[1] == 160
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


-----WARNING: Model s01 failed to learn: sum(ans)=480.0 sum(y)=288.0 len(y)=480
	AROUSAL-s01|Acc=0.6|10-CV=0.6|STD=0.0|Time spend=1.3191819190979004


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s02|Acc=0.63542|10-CV=0.60833|STD=0.01394|Time spend=1.6479735374450684


...

	AROUSAL-s30|Acc=0.94167|10-CV=0.93583|STD=0.01669|Time spend=0.7009027004241943


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s31|Acc=0.59167|10-CV=0.55083|STD=0.03343|Time spend=0.7258539199829102


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s32|Acc=0.85417|10-CV=0.835|STD=0.02409|Time spend=0.6284253597259521
AROUSAL|Acc=0.7819661458333335|10-CV=0.7668750000000002|STD=0.0183199131872165


In [21]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape[1] == 160
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s01|Acc=0.79583|10-CV=0.8|STD=0.02838|Time spend=0.7523384094238281


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s02|Acc=0.99792|10-CV=0.99833|STD=0.00333|Time spend=0.815838098526001


...

	VALENCE-s30|Acc=0.89375|10-CV=0.8875|STD=0.01455|Time spend=0.8396103382110596


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


-----WARNING: Model s31 failed to learn: sum(ans)=480.0 sum(y)=276.0 len(y)=480
	VALENCE-s31|Acc=0.575|10-CV=0.575|STD=0.0|Time spend=0.7985196113586426


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s32|Acc=0.63958|10-CV=0.63167|STD=0.03287|Time spend=0.9344546794891357
VALENCE|Acc=0.7524739583333334|10-CV=0.7381770833333334|STD=0.02200093042823058


In [22]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.7672200520833331|10-CV=0.7525260416666669|STD=0.020160421807723553
